In [1]:
# Tutorial source: https://nbviewer.jupyter.org/github/krasserm/face-recognition/blob/master/face-recognition.ipynb?flush_cache=true

In [8]:
from keras.models import load_model
from keras.utils import CustomObjectScope

import tensorflow as tf

In [10]:
with CustomObjectScope({'tf': tf}):
    model = load_model('./model/nn4.small2.v2.h5')

ValueError: bad marshal data (unknown type code)